## Preprocessing

In [5]:
!pip install tensorflow==2.11.*

  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached tensorboard-2.11.2-py3-none-any.whl (6.0 MB)
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\msfte\\anaconda3\\envs\\PythonData\\Lib\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



In [6]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

ModuleNotFoundError: No module named 'tensorflow.python.eager.polymorphic_function'

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
appl_df = application_df.drop(['EIN', 'NAME'], axis=1)
appl_df.head()

In [ ]:
# Determine the number of unique values in each column.
appl_df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_ct = appl_df['APPLICATION_TYPE'].value_counts()
app_ct

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_ct[app_ct < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    appl_df['APPLICATION_TYPE'] = appl_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
appl_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_ct = appl_df['CLASSIFICATION'].value_counts()
class_ct

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#class_ct = appl_df['CLASSIFICATION'].value_counts(counts > 1)

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_ct[class_ct < 250].index)

# Replace in dataframe
for cls in classifications_to_replace:
    appl_df['CLASSIFICATION'] = appl_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
appl_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
charity_numbers = pd.get_dummies(appl_df)
charity_numbers

In [ ]:
# Split our preprocessed data into our features and target arrays
y = charity_numbers['IS_SUCCESSFUL']
X = charity_numbers.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Import an Adaptive Boosting classifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
clf = AdaBoostClassifier(random_state=1).fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

print(classification_report(y_test, y_pred))
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

## Compile, Train and Evaluate the Model

I adjusted the hidden layers a few times, from 2 layers @ 8/16, to 3 layers 11/15/22. The additional layers produced a better outcome in terms of accuracy, but still not the 75% I hoped for.

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  11
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 22

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer1, activation="relu", input_dim=45))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer3, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

In [ ]:
#!pip install tf-nightly

I attempted to use the Adadelta and AdamW optimization here as well; I wanted to address weight decay and both do this. However, the AdamW is experimental and when I pipinstalled the module for applying this optimization, it gave me an error that wasn't resolvable. When I attempted the Adadelta optimizer, the model immediately fit to a 0.5592 loss: 0.7433 accuracy throughout every single epoch, which clearly wasn't the expected outcome, so I discarded that optimize and went back to the Adam optimizer instead. ML is iterative and requires a considerable amount of trial-and-error.

In [ ]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

So far, I have attempted to adjust the epochs 5 times: 50, 100, 200, 300, 250. 200 is probably optimal; anything above this seems to create decay.

In [ ]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharityboosted.h5')

In [ ]:
# save charity_numbers as a .csv
charity_numbers.to_csv('charity_numbers_boosted.csv')